# Konserveerunud 1000 juh ennekui  lauset

Loeme failist 10 000 juhuslikku lauset

In [1]:
import csv
import pandas as pd
import json

In [2]:
allsentences = f'result/1_sentences_with_advmod.tsv'

df = pd.read_csv(allsentences, sep='\t', header=None, quoting=csv.QUOTE_NONE)  
df.shape

(10626918, 3)

In [3]:
df.head()

,0,1,2
0,0,69,"Endine punkar , tulevane lendur , karateka ja ..."
1,0,262,"Bändi , mille solist ei osanud laulda , kidram..."
2,0,739,Kui sageli tuli sul kooli juhtkonna ees aru an...
3,0,791,"Mind kutsuti tihti direktori jutule , aga mitt..."
4,0,1080,"Tol ajal oli mu ainuke prioriteet muusika , mi..."


Ajame read juhuslikku järjekorda

In [4]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,0,1,2
0,628347,697,"Seega oleks vaja ligikaudu 2 , 3-2 , 8 miljard..."
1,395914,2726,"Ühiselt ütlevad õpetaja ja õpilane , et maailm..."
2,345435,2361,""" Seni kasutatakse üle-eestilist skeemi , ürit..."
3,102060,2685,"Tegemist on väikeste investeeringutega , mis p..."
4,496935,329,Siiski läks Berliini millenniumivahetus mõnes ...


### 1000  juhuslikku enne_kui lauset

Laseme süntaksi peale
Eemaldame admvod koos selle alampuuga
Kontrollime, kas süntaks on konserveerunud

In [5]:
import stanza
import os
from estnltk import Text
from estnltk.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger
from estnltk.converters.conll_exporter import  sentence_to_conll
import numpy as np


In [6]:
model_path = ".../estnltk-version_1.6/estnltk/taggers/syntax/stanza_tagger/stanza_resources"
model_path = '/Users/rabauti/stanza-models/stanza_recources'
input_type="morph_extended"
#stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, resources_path=model_path)
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)


In [7]:

#tekst on tokeniseeritud, võib tühikuga otsida küll
def is_enne_kui_examples(text):
    sent = ' ' + text.lower() + ' '
    if " rohkem kui " in sent or " vähem kui " in sent or " samapalju kui " in sent:
        return True
    return False

def write_to_file(t, f):
    f.write(t)

def get_children(span, lst):
    if not span.children:
        return
    else:
        for child in span.children:
            lst += [int(child.id)-1]
            get_children(child, lst)


def remove_deprel(sentence, dprl, stanza_tagger):
    changes = {}
    no_rem_dprls = 0
   
    #Lausete lühendamine
    sent = sentence.sentences.text
    len_sent = len(sent)
    lsts = []
    for span in sentence.stanza_syntax:
        if span.deprel == dprl:
            lst = [int(span.id)-1]
            get_children(span, lst)
            lsts += lst

    sent = np.array(sent)
    sent[list(set(lsts))] = ":::"
    sent = list(sent)
    sent = list(filter(lambda a: a != ":::", sent))
    if  len(sent) != len_sent:
       
        no_rem_dprls += (len_sent - len(sent))
    
    short_sent = " ".join(sent)
    short_sent = short_sent[0].upper() + short_sent[1:]
    short_sent_txt = Text(short_sent)
    short_sent_txt.analyse('all')
    stanza_tagger.tag(short_sent_txt)
    len_short_sent = len(sent)
    
    pikkus = 0
    e = 0
    m = 0

    sent_deprel = ''
    changed = False
    while len_sent > pikkus:
        sp_long = sentence.stanza_syntax[pikkus]
        if  e < len_short_sent:
            sp_short = short_sent_txt.stanza_syntax[e]
        
        if sp_short.text.lower() == sp_long.text.lower() and pikkus == e + m and e < len_short_sent:
            sent_deprel += sp_short["deprel"] + " "
            if sp_short["deprel"] != sp_long.deprel:
                changed = True
                key = sp_long.deprel + "_" + sp_short["deprel"]
                if key not in changes.keys():
                    changes[key] = 1
                else:
                    changes[key] += 1
            e += 1
        else:
            sent_deprel += "_" + " "
            m += 1
        pikkus += 1     
    
    # changed - kas muutus
    # 
    # short_sent_txt.stanza_syntax
    # no_rem_dprls, changes
    return {'changed': changed \
                , 'short_sent_txt': short_sent_txt \
                , 'no_rem_dprls': no_rem_dprls \
                , 'changes': changes \
                , 'sent_deprel': sent_deprel}
   
    

In [10]:
%%time

df = df.sample(frac=1).reset_index(drop=True)
df.head()

dprl = "advmod"

RESULTS_PATH = "./result/"


outChangedSentencesTSV = RESULTS_PATH+"3_enne_kui_changed_sentences_" + dprl + ".tsv"
outConservedSentencesTSV = RESULTS_PATH+"3_enne_kui_conserved_sentences_" + dprl + ".tsv"

outChanged = open(outChangedSentencesTSV, 'w', encoding='utf-8')
outConserved = open(outConservedSentencesTSV, 'w', encoding='utf-8')

# create the csv writer
wChanged = csv.writer(outChanged, delimiter='\t')
wConserved = csv.writer(outConserved, delimiter='\t')


count = 1000

# deprel to remove

sentences_count = 0
syntax_conserved = 0
syntax_changed = 0

all_changes = {}

wConserved.writerow(('COLLECTION', 'SPANSTART', 'TYPE', 'TEXT', ),)
wChanged.writerow(('COLLECTION', 'SPANSTART', 'TYPE', 'TEXT', ),)

emptyrow = ('', '', '', '', )
for ind in range(df.shape[0]):
    
    
    proc_txt_no = df[0][ind]
    sent_span_start = df[1][ind]
     
    txt = df[2][ind]
    #print (txt)
    
    #ignoreerime lauseid, mis pole enne_kui
    if not is_enne_kui_examples(txt): continue
        
    
    sent = stanza_tagger.tag(Text(txt).analyse("all"))
    
    #ignoreerime lauseid, kus pole advmod'i siiski sees
    if not dprl in sent.deprel:  continue
    
    
    sentences_count += 1
    result = remove_deprel(sent, dprl, stanza_tagger)
    
    if not len(result['changes']):
        syntax_conserved +=1
    else:
        syntax_changed +=1
    all_changes = {k: all_changes.get(k, 0) + result['changes'].get(k, 0) for k in set(all_changes) | set(result['changes'])}

    row1 = [str(proc_txt_no) \
                , str(sent_span_start) \
                , 'O'
                , sent.text ]
    
    row2 = [str(proc_txt_no) \
                , str(sent_span_start) \
                , 'S'
                , result['short_sent_txt'].text]
    
    if result['changed']:
        wChanged.writerow(row1)
        wChanged.writerow(row2)
        wChanged.writerow(emptyrow)
    else:
        wConserved.writerow(row1)
        wConserved.writerow(row2)
        wConserved.writerow(emptyrow)

    if syntax_changed >= count: break 

outChanged.close()
outConserved.close()
print (f'Teisendatud {sentences_count} lauset.')
print (f'Kokku vaadati {ind+1} lauset.')
print (f'Süntaks konserveerunud {syntax_conserved} lausel.')
print (f'Süntaks ei konserveerunud {syntax_changed} lausel.')

print (all_changes)

  

Teisendatud 2439 lauset.
Kokku vaadati 505225 lauset.
Süntaks konserveerunud 1439 lausel.
Süntaks ei konserveerunud 1000 lausel.
{'nsubj_amod': 1, 'obl_nmod': 74, 'root_nsubj:cop': 6, 'compound:prt_obl': 1, 'advcl_amod': 2, 'conj_flat': 5, 'obj_case': 2, 'cop_csubj': 1, 'advcl_cop': 1, 'amod_nsubj': 1, 'fixed_mark': 8, 'acl:relcl_cop': 1, 'nmod_obj': 24, 'obl_compound:prt': 1, 'acl_aux': 3, 'obl_root': 23, 'obl_csubj': 1, 'root_nummod': 2, 'conj_cop': 1, 'ccomp_conj': 8, 'nsubj_case': 2, 'appos_obj': 1, 'root_conj': 23, 'advcl_ccomp': 16, 'obl_obj': 20, 'conj_parataxis': 5, 'vocative_nsubj': 1, 'advcl_obj': 24, 'nsubj_appos': 6, 'root_csubj:cop': 1, 'acl_xcomp': 9, 'nsubj:cop_conj': 4, 'conj_nsubj': 5, 'advcl_root': 27, 'conj_advmod': 27, 'aux_csubj:cop': 1, 'compound_nummod': 3, 'root_advmod': 22, 'compound:prt_case': 5, 'nmod_appos': 8, 'appos_obl': 3, 'nummod_nsubj': 2, 'cop_acl': 4, 'nummod_compound': 1, 'csubj:cop_csubj': 1, 'obl_case': 3, 'advcl_compound:prt': 2, 'obj_xcomp': 1, 

In [23]:

dfChanged = pd.read_csv(outChangedSentencesTSV, sep='\t', quotechar = '"')  
dfChanged.shape

(3000, 4)

In [24]:
dfChanged.head()

,COLLECTION,SPANSTART,TYPE,TEXT
0,482428.0,472.0,O,"Selsamal ajaleheküljel on rohkem kui üks kord jutuks tulnud see , et lause või sõnad , mida peaminister või mõni teine riigitegelane välja ütles , ei väljendanud seda , mida ta mõtles ja öelda tahtis ."
1,482428.0,472.0,S,"Selsamal ajaleheküljel on kui üks kord jutuks tulnud see , et lause või sõnad , mida peaminister või mõni teine riigitegelane välja ütles , ei väljendanud seda , mida ta mõtles ja öelda tahtis ."
2,NaN,NaN,NaN,NaN
3,222666.0,608.0,O,Rohkem kui kolmandik Tai 76 provintsist on kantud linnugripi riskitsooni .
4,222666.0,608.0,S,Kolmandik Tai 76 provintsist on kantud linnugripi riskitsooni .
